In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn import preprocessing as ps
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

In [ ]:
od.download(
    "https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data"
)

Skipping, found downloaded files in "./house-prices-advanced-regression-techniques" (use force=True to force download)


In [ ]:
train=pd.read_csv("/content/house-prices-advanced-regression-techniques/train.csv")
test=pd.read_csv("/content/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
null_keys_train=[k for k,i in train.isnull().sum().items() if i!=0]
null_keys_train

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [ ]:
null_keys_test=[k for k,i in test.isnull().sum().items() if i!=0]
null_keys_test

['MSZoning',
 'LotFrontage',
 'Alley',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType']

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
oe=OrdinalEncoder()
non_numeric_features=train.select_dtypes(exclude='number').columns.tolist()
for i in non_numeric_features:
  train[i]=oe.fit_transform(train[[i]])
  test[i]=oe.fit_transform(test[[i]])

In [ ]:
mssi=KNNImputer(n_neighbors=5, weights="uniform")
for i in null_keys_train:
  train[i]=mssi.fit_transform(train[[i]])
for i in null_keys_test:
  test[i]=mssi.fit_transform(test[[i]])

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   float64
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   float64
 6   Alley          1460 non-null   float64
 7   LotShape       1460 non-null   float64
 8   LandContour    1460 non-null   float64
 9   Utilities      1460 non-null   float64
 10  LotConfig      1460 non-null   float64
 11  LandSlope      1460 non-null   float64
 12  Neighborhood   1460 non-null   float64
 13  Condition1     1460 non-null   float64
 14  Condition2     1460 non-null   float64
 15  BldgType       1460 non-null   float64
 16  HouseStyle     1460 non-null   float64
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
y=train['SalePrice']
train=train.drop(['SalePrice','Id'],axis=1)
id=test['Id']
test=test.drop(['Id'],axis=1)
normal=ps.MinMaxScaler(feature_range=(0, 1))

for i in train.keys():
  train[i]=normal.fit_transform(np.array(train[i]).reshape(-1,1))

for i in test.keys():
  test[i]=normal.fit_transform(np.array(test[i]).reshape(-1,1))

In [ ]:
pca = PCA(n_components=8)
train = pca.fit_transform(train)
test=pca.fit_transform(test)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(train,y,test_size=0.1,random_state=0)

In [ ]:
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(8)),
    keras.layers.Dense(16, kernel_initializer='normal',activation='relu'),
    keras.layers.Dense(16, kernel_initializer='normal',activation='relu'),
    keras.layers.Dense(8, kernel_initializer='normal',activation='relu'),
    keras.layers.Dense(1, kernel_initializer='normal'),
])

model.summary()

Model: "sequential_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_337 (Dense)           (None, 16)                144       
                                                                 
 dense_338 (Dense)           (None, 16)                272       
                                                                 
 dense_339 (Dense)           (None, 8)                 136       
                                                                 
 dense_340 (Dense)           (None, 1)                 9         
                                                                 
Total params: 561 (2.19 KB)
Trainable params: 561 (2.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss=keras.losses.MeanSquaredError(),
              optimizer=keras.optimizers.Adam(learning_rate=0.35),
              )

In [ ]:
model.fit(
    x_train,
    y_train,
    batch_size=1,
    epochs=100,
    shuffle=True,
    verbose=2)

Epoch 1/100
1314/1314 - 4s - loss: 2776043776.0000 - 4s/epoch - 3ms/step
Epoch 2/100
1314/1314 - 2s - loss: 2808617728.0000 - 2s/epoch - 2ms/step
Epoch 3/100
1314/1314 - 2s - loss: 2780494592.0000 - 2s/epoch - 2ms/step
Epoch 4/100
1314/1314 - 2s - loss: 2681106432.0000 - 2s/epoch - 2ms/step
Epoch 5/100
1314/1314 - 3s - loss: 2866887680.0000 - 3s/epoch - 2ms/step
Epoch 6/100
1314/1314 - 2s - loss: 2520127488.0000 - 2s/epoch - 2ms/step
Epoch 7/100
1314/1314 - 2s - loss: 2848876544.0000 - 2s/epoch - 2ms/step
Epoch 8/100
1314/1314 - 2s - loss: 2621608704.0000 - 2s/epoch - 2ms/step
Epoch 9/100
1314/1314 - 2s - loss: 2674074368.0000 - 2s/epoch - 2ms/step
Epoch 10/100
1314/1314 - 3s - loss: 2717814272.0000 - 3s/epoch - 2ms/step
Epoch 11/100
1314/1314 - 3s - loss: 2582614272.0000 - 3s/epoch - 2ms/step
Epoch 12/100
1314/1314 - 2s - loss: 2647271424.0000 - 2s/epoch - 2ms/step
Epoch 13/100
1314/1314 - 3s - loss: 2697586176.0000 - 3s/epoch - 2ms/step
Epoch 14/100
1314/1314 - 2s - loss: 2854427648.

In [ ]:
model.evaluate(x_test, y_test, batch_size=1,verbose=2)

146/146 - 0s - loss: 2320675840.0000 - 300ms/epoch - 2ms/step


2320675840.0

In [ ]:
result=model.predict(test)
result

46/46 [==============================] - 0s 2ms/step


array([[124420.086],
       [124420.086],
       [208441.31 ],
       ...,
       [133866.67 ],
       [124420.086],
       [197656.33 ]], dtype=float32)

In [ ]:
answer=pd.DataFrame({
    'Id':np.array(id),
    'SalePrice':result.flatten()
})
answer.to_csv('./answer.csv',sep=',',index=False,header=True)